# Examination of effecient NHL roster design

## purpose of notebook:

- import all csv files from the 2010 season
- categorize and merge all csv files to create final data frame
- remove irrelavant observations
- keep the first two games of the season for analysis
- store final data frame

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy

## import data frames

- import play by play csv file

In [2]:
dm = pd.read_csv('t_play_by_play_o.csv')
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])
dm = dm [['Season', 'GameNumber', 'EventNumber', 'Period', 'AdvantageType', 'EventTimeFromZero', 'EventTimeFromTwenty', 'EventType', 'EventDetail', 'VPlayer1', 'VPlayer1Position', 'VPlayer2', 'VPlayer2Position', 'VPlayer3', 'VPlayer3Position', 'VPlayer4', 'VPlayer4Position', 'VPlayer5', 'VPlayer5Position', 'VPlayer6', 'VPlayer6Position', 'HPlayer1', 'HPlayer1Position', 'HPlayer2', 'HPlayer2Position', 'HPlayer3', 'HPlayer3Position', 'HPlayer4', 'HPlayer4Position', 'HPlayer5', 'HPlayer5Position', 'HPlayer6', 'HPlayer6Position']]

- import play by play giveaway csv file and rename columns

In [3]:
dg = pd.read_csv('t_play_by_play_giveaway_detail_o.csv')
dg = dg.rename(columns={'GivePlayerNumber': 'PlayerNumber', 'GivePlayerLName': 'PlayerName', 'GiveTeamCode': 'TeamCode'})

- import play by play takeaway csv file and rename columns

In [4]:
dt = pd.read_csv('t_play_by_play_takeaway_detail_o.csv')
dt = dt.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dt = dt.rename(columns={'TakePlayerNumber': 'PlayerNumber', 'TakePlayerLName': 'PlayerName', 'TakeTeamCode': 'TeamCode'})

- import goal detail csv file and rename column

In [5]:
dl = pd.read_csv('t_play_by_play_goal_detail_o.csv')
dl = dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dl = dl.rename(columns={'PlayerLName': 'PlayerName'})

- import play by play shot detail csv file and rename column

In [6]:
ds = pd.read_csv('t_play_by_play_shot_detail_o.csv')
ds = ds.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
ds = ds.rename(columns={'PlayerLName': 'PlayerName'})

- import play by play miss detail csv file and rename column

In [7]:
dn = pd.read_csv('t_play_by_play_miss_detail_o.csv')
dn = dn.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dn = dn.rename(columns={'PlayerLName': 'PlayerName'})

- import play by play block detail csv file and rename columns

In [8]:
db = pd.read_csv('t_play_by_play_block_detail_o.csv')
db = db.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
db = db.rename(columns={'ShotPlayerLName': 'ShotPlayerName', 'BlockPlayerLName': 'PlayerName', 'BlockPlayerNumber': 'PlayerNumber', 'BlockTeamCode': 'TeamCode'})

- import play by play hit detail csv file and rename columns

In [9]:
dh = pd.read_csv('t_play_by_play_hit_detail_o.csv')
dh = dh.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dh = dh.rename(columns={'HitterPlayerLName': 'HitterPlayerName', 'HitteePlayerLName': 'HitteePlayerName'})

 - import play by play faceoff detail csv file and rename columns

In [10]:
df = pd.read_csv('t_play_by_play_faceoff_detail_o.csv')
df = df.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
df = df.rename(columns={'VPlayerLName': 'VPlayerName', 'HPlayerLName': 'HPlayerName'})

 - import play by play penalty detail csv file and rename columns

In [11]:
dp = pd.read_csv('t_play_by_play_penalty_detail_o.csv')
dp = dp.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dp = dp.rename(columns={'PenaltyPlayerLName': 'PenaltyPlayerName', 'DrawnByPlayerLName': 'DrawnByPlayerName'})

- import game detail csv file 

In [12]:
dd = pd.read_csv('t_game_detail_o.csv')

- import scoring csv file and rename columns

In [13]:
dc = pd.read_csv('t_scoring_detail_o.csv')
dc['GoalTime'] = dc['Time'].copy()
dc = dc.rename(columns={'Time': 'EventTimeFromZero', 'GoalType': 'AdvantageType'})
dc = dc [['Season', 'GameNumber', 'GoalNumber', 'Period', 'GoalTime', 'EventTimeFromZero', 'AdvantageType', 'TeamCode', 'Assist1Player', 'Assist2Player']]

## group and merge data frames based on similarity

Since giveaway (dg) and takeaway (dt) csv files have identical columns, they are merged together. Thus, dt is merged on dg:

## merge giveaways and takeaways 

In [14]:
dg = dg.merge(dt, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone'], how='outer')
dg.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

dg now contains observations from both type of events. The merged data frame is ascendingly sorted by season, game number and event number.

## merge goal,  shot,  miss and block dataframes

Goal, miss and block are result of a shot. For that reason, they are merged together. 

- merge shot data frame (ds) onto goal data frame (dl):

In [16]:
dl = dl.merge(ds, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone', 'ShotType', 'Length'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)


**dl** now contains observations from both shot and goal data set. The merged data frame is ascendingly sorted by season, game number and event number.

- merge miss dataframe (dm) onto goal data frame (dl): 

In [18]:
dl = dl.merge(dn, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone', 'ShotType', 'Length'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

**dl** now contains observations from goal, shot and miss events. The merged data frame is ascendingly sorted by season, game number and event number.

- merge block (db) onto goal dataframe (dl):

In [20]:
dl = dl.merge(db, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone', 'ShotType'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

**dl** now contains observations from goal, shot, miss and block events. The merged data frame is ascendingly sorted by season, game number and event number.

In [21]:
dl.columns

Index(['GameNumber', 'EventNumber', 'TeamCode', 'PlayerNumber', 'PlayerName',
       'ShotType', 'Zone', 'Length', 'Season', 'ShotResult', 'ShotTeamCode',
       'ShotPlayerNumber', 'ShotPlayerName'],
      dtype='object')

## merge faceoff, hit and penalty together

- merge hit data (dh) frame onto faceoff data frame (df):

In [22]:
df = df.merge(dh, on=['Season', 'GameNumber', 'EventNumber', 'Zone'], how='outer')
df.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

**df** now contains observations from both faceoff and hit events. The merged data frame is ascendingly sorted by season, game number and event number.

- merge penalty data frame (dh) onto faceoff data frame (df):

In [24]:
df = df.merge(dp, on=['Season', 'GameNumber', 'EventNumber', 'Zone'], how='outer')
df.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

**df** now contains observations from faceoff, hit and penalty events. The merged data frame is ascendingly sorted by season, game number and event number.

## merge merged data frames

Three merged data frames have been created:  dg, dl and dg. The next step is to merge the merged data frames together.

- merge dg onto dl:

In [26]:
dl = dl.merge(dg, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)


**dl** now contains observations from goal, shot, miss , block, giveaway and takeaway events. The merged data frame is ascendingly sorted by season, game number and event number.

- merge df onto dl:

In [28]:
dl = dl.merge(df, on=['Season', 'GameNumber', 'EventNumber', 'Zone'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)


**dl** now contains observations from goal, shot, miss , block, giveaway, takeaway, faceoff, hit and penalty events. The merged data frame is ascendingly sorted by season, game number and event number.

## merge detail play (dd) onto all ice events (dl)

In [30]:
dl = dl.merge(dd, on=['Season', 'GameNumber', 'VTeamCode', 'HTeamCode'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [31]:
dl.columns

Index(['GameNumber', 'EventNumber', 'TeamCode', 'PlayerNumber', 'PlayerName',
       'ShotType', 'Zone', 'Length', 'Season', 'ShotResult', 'ShotTeamCode',
       'ShotPlayerNumber', 'ShotPlayerName', 'WinTeamCode', 'VTeamCode',
       'VPlayerNumber', 'VPlayerName', 'HTeamCode', 'HPlayerNumber',
       'HPlayerName', 'HitterTeamCode', 'HitterPlayerNumber',
       'HitterPlayerName', 'HitteeTeamCode', 'HitteePlayerNumber',
       'HitteePlayerName', 'PenaltyTeamCode', 'PenaltyPlayerNumber',
       'PenaltyPlayerName', 'PenaltyType', 'DrawnByTeamCode',
       'DrawnByPlayerNumber', 'DrawnByPlayerName', 'GameDate'],
      dtype='object')

## merge all on-ice events (dl) onto play by play (dm)

The merged data frame containing all on-ice events (dl) is merged onto the play by play data frame by season, game number and event number.

In [32]:
dm = dm.merge(dl, on=['Season', 'GameNumber', 'EventNumber'], how='outer')
dm.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

## merge scoring detail (dc) onto play by play (dm)

The final merge is to add scoring detail data frame (dc) onto play by play data frame (dm). The purpose is to include assist 1 player name, assist 1 player number, assist 2 player number and assist player 2 name into the data frame that will be used for player evaluation analysis.

In [34]:
dm = dm.merge(dc, on=['Season', 'GameNumber', 'Period', 'EventTimeFromZero', 'AdvantageType', 'TeamCode'], how='outer')

## remove irrelevant observations


Tv time-out, goalie time-out and icing are listed as "stoppage" events and are removed from the data frame as they have no impact on the probability of a goal being scored.

In [35]:
dm = dm[dm['EventType']!='STOP']

## man-advantage scenarios

In [36]:
value_list = ['PP', 'SH']
dm[dm['AdvantageType'].isin(value_list)]
dm = dm[dm['AdvantageType'] != 'PP']
dm = dm[dm['AdvantageType'] != 'SH']
dm['AdvantageType'] = dm['AdvantageType'].fillna('EV')

Since the player evaluation model uses only even strength situations, man-advantage scenarios are dropped from the data frame

## keep first two games of the season

To conduct analysis for both player evalution model and roster design model, the first two game of the 2010 season are used. 

In [37]:
dm = dm[dm['GameNumber'] <= 20002]

In [38]:
dm

,Season,GameNumber,EventNumber,Period,AdvantageType,EventTimeFromZero,EventTimeFromTwenty,EventType,EventDetail,VPlayer1,...,PenaltyPlayerName,PenaltyType,DrawnByTeamCode,DrawnByPlayerNumber,DrawnByPlayerName,GameDate,GoalNumber,GoalTime,Assist1Player,Assist2Player
0,2010,20001,1.0,1,EV,0,1200.0,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,11.0,...,NaN,NaN,NaN,NaN,NaN,2010-10-07,NaN,NaN,NaN,NaN
1,2010,20001,3.0,1,EV,15,1185.0,HIT,"TOR #37 BRENT HIT MTL #26 GORGES, Off. Zone",11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,20001,4.0,1,EV,46,1154.0,HIT,"MTL #14 PLEKANEC HIT TOR #2 SCHENN, Off. Zone",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,20001,5.0,1,EV,57,1143.0,HIT,"MTL #76 SUBBAN HIT TOR #15 KABERLE, Neu. Zone",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,20001,6.0,1,EV,69,1131.0,GIVE,"TOR&nbsp;GIVEAWAY - #35 GIGUERE, Def. Zone",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010,20001,7.0,1,EV,73,1127.0,BLOCK,"MTL #76 SUBBAN BLOCKED BY TOR #2 SCHENN, Wris...",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2010,20001,8.0,1,EV,86,1114.0,SHOT,"MTL ONGOAL - #81 ELLER, Wrist, Off. Zone, 11 ft.",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2010,20001,9.0,1,EV,91,1109.0,SHOT,"MTL ONGOAL - #46 KOSTITSYN, Snap, Off. Zone, 8...",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2010,20001,10.0,1,EV,95,1105.0,BLOCK,"MTL #76 SUBBAN BLOCKED BY TOR #32 VERSTEEG, S...",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2010,20001,11.0,1,EV,102,1098.0,MISS,"MTL #76 SUBBAN, Slap, Wide of Net, Off. Zone, ...",15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# store final data frame

The merged play by play data frame is stored 

In [39]:
dm.to_csv('pbpmerge.csv', index='False', sep=',')

The next step is to conduct the player evaluation model, which will take place in the "roster_design_player_evalution" notebook.